# Import libraries

In [1]:
import numpy as np
from numpy.random import RandomState
import numpy.ma as ma

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
%matplotlib inline

import h5py
import ot
from numpy.random import Generator, PCG64
from sklearn import metrics
import itertools

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.neighbors import KNeighborsClassifier
from tqdm import tqdm

In [2]:
sigAliasList    = ['sig_A', 'sig_h0', 'sig_hch', 'sig_LQ']
sigFilenameList = ['Ato4l_lepFilter_13TeV_filtered.h5', 'hToTauTau_13TeV_PU20_filtered.h5', 'hChToTauNu_13TeV_PU20_filtered.h5', 'leptoquark_LOWMASS_lepFilter_13TeV_filtered.h5']

In [3]:
#-- Set base directory and data directory path --#
basePath   = './'
dataPath   = 'Data/'

bkgPath    = basePath+dataPath+'background_for_training.h5'
sigPathList = []
for x in sigFilenameList:
  sigPathList.append(basePath+dataPath+x)

# Functions

In [4]:
%run centralFunctions.ipynb

# Loading Data

In [5]:
dataDict = {}
dataDict['bkg'] = h5py.File(bkgPath, 'r')

for i in range(len(sigAliasList)):
  alias   = sigAliasList[i]
  sigPath = sigPathList[i]
  dataDict[alias] = h5py.File(sigPath, 'r')

In [6]:
bkg_data = dataDict['bkg']['Particles'][:,:,0:3]
sig_data = {}

for alias in sigAliasList:
  sig_data[alias] = dataDict[alias]['Particles'][:,:,0:3]

# Low $p_T$ range

In [7]:
nEvents = 100
random_state = Generator(PCG64(123))
OTSCHEME = {}
OTSCHEME['normPT'] = True
OTSCHEME['balanced'] = True
OTSCHEME['noZeroPad'] = False
OTSCHEME['individualOT'] = False

In [8]:
total_event_pT = {}

total_event_pT['bkg'] = np.sum(bkg_data[:, :, 0], axis=1)

for alias in sigAliasList:
    total_event_pT[alias] = np.sum(sig_data[alias][:,:,0], axis=1)

pTrange = [0,50,100,150,200,500,1000]
neighbor_list = list(range(5, 400,10))
avg_aucs = []
std_aucs = []
avg_ks = []
std_ks = []
for i in range(0, len(pTrange)-1):
    lower_bound = pTrange[i]
    upper_bound = pTrange[i+1]
    
    filtered_events = {}
    mask = (total_event_pT['bkg'] >= lower_bound) & (total_event_pT['bkg'] <= upper_bound)
    
    filtered_events['bkg'] = randomDataSample(bkg_data[mask],nEvents,random_state)
    np.random.seed(i)
    permutation = np.random.permutation(nEvents*2)
    
    for alias in sigAliasList:
        mask = (total_event_pT[alias] >= lower_bound) & (total_event_pT[alias] <= upper_bound)
        filtered_events[alias] = randomDataSample(sig_data[alias][mask],nEvents,random_state)
        
        event_list = np.concatenate((filtered_events['bkg'],filtered_events[alias]))
        event_labels = np.asarray([0] * nEvents + [1] * nEvents)
        event_list = event_list[permutation]
        event_labels = event_labels[permutation]
        
        distance_matrix = calcOTDistance(event_list, event_list, OTSCHEME, '2D', Matrix = True)
        
        avg_auc, std_auc, avg_k, std_k = kNN_cross_validation(distance_matrix, event_labels, neighbor_list, k_fold=5)
        print(avg_auc, std_auc, avg_k, std_k)
        avg_aucs.append(avg_auc)
        std_aucs.append(std_auc)
        avg_ks.append(avg_k)
        std_ks.append(std_k)

Fitting Models: 100%|██████████| 40/40 [00:00<00:00, 69.70it/s]


0.6193241561554373 0.013909554559565948 25.0 12.649110640673518


Fitting Models: 100%|██████████| 40/40 [00:00<00:00, 75.99it/s]


0.5560362022323223 0.026013491872497616 85.0 87.40709353364863


Fitting Models: 100%|██████████| 40/40 [00:00<00:00, 73.63it/s] 


...

0.5829378691399614 0.021238964288079334 83.0 63.686733312362634


Fitting Models: 100%|██████████| 40/40 [00:00<00:00, 76.12it/s] 


0.6202817253317441 0.0341775808408001 81.0 74.45804187594514


Fitting Models: 100%|██████████| 40/40 [00:00<00:00, 76.53it/s] 


0.5284963612145445 0.015364811569643429 57.0 80.34923770640266


In [9]:
print(avg_aucs)
print(std_aucs)

[0.6193241561554373, 0.5560362022323223, 0.5752272736176252, 0.6840947062389287, 0.797380753997553, 0.6761395554738507, 0.607118567366921, 0.7468665866068318, 0.7011452113981134, 0.5667354772174112, 0.6015605971417531, 0.6883180348051865, 0.6251299810860171, 0.5761864516416402, 0.6665967908890511, 0.6981814488486944, 0.6258593862764383, 0.5841808994695518, 0.6885509588655917, 0.6166094819906541, 0.798913933102407, 0.5829378691399614, 0.6202817253317441, 0.5284963612145445]
[0.013909554559565948, 0.026013491872497616, 0.037555171735308014, 0.03150385066000139, 0.008993921054521814, 0.035688653156475125, 0.026939186162531433, 0.021490142570251385, 0.005785647854299646, 0.04953517731908581, 0.03723336061553597, 0.015572400289487753, 0.020683095471606176, 0.02572489625664325, 0.021584085318690237, 0.019362046157130636, 0.024167621160884223, 0.006719364053971755, 0.020427488648409705, 0.020159541926277634, 0.02230500501928155, 0.021238964288079334, 0.0341775808408001, 0.015364811569643429]


In [10]:
grouped_data_1 = [avg_aucs[i:i+4] for i in range(0, len(avg_aucs), 4)]
grouped_data_2 = [std_aucs[i:i+4] for i in range(0, len(std_aucs), 4)]
print(grouped_data_1)
print(grouped_data_2)

[[0.6193241561554373, 0.5560362022323223, 0.5752272736176252, 0.6840947062389287], [0.797380753997553, 0.6761395554738507, 0.607118567366921, 0.7468665866068318], [0.7011452113981134, 0.5667354772174112, 0.6015605971417531, 0.6883180348051865], [0.6251299810860171, 0.5761864516416402, 0.6665967908890511, 0.6981814488486944], [0.6258593862764383, 0.5841808994695518, 0.6885509588655917, 0.6166094819906541], [0.798913933102407, 0.5829378691399614, 0.6202817253317441, 0.5284963612145445]]
[[0.013909554559565948, 0.026013491872497616, 0.037555171735308014, 0.03150385066000139], [0.008993921054521814, 0.035688653156475125, 0.026939186162531433, 0.021490142570251385], [0.005785647854299646, 0.04953517731908581, 0.03723336061553597, 0.015572400289487753], [0.020683095471606176, 0.02572489625664325, 0.021584085318690237, 0.019362046157130636], [0.024167621160884223, 0.006719364053971755, 0.020427488648409705, 0.020159541926277634], [0.02230500501928155, 0.021238964288079334, 0.0341775808408001,

In [8]:
total_event_pT_bkg = np.sum(bkg_data[:, :, 0], axis=1)
mask = total_event_pT_bkg <= 50
low_pT_events_bkg = randomDataSample(bkg_data[mask],nEvents, random_state)

In [9]:
total_event_pT_A = np.sum(sig_data['sig_A'][:,:,0],axis = 1)
mask = total_event_pT_A <= 50
low_pT_events_A = randomDataSample(sig_data['sig_A'][mask],nEvents, random_state)

In [10]:
total_event_pT_h0 = np.sum(sig_data['sig_h0'][:,:,0],axis = 1)
mask = total_event_pT_h0 <= 50
low_pT_events_h0 = randomDataSample(sig_data['sig_h0'][mask],nEvents, random_state)

In [11]:
total_event_pT_hch = np.sum(sig_data['sig_hch'][:,:,0],axis = 1)
mask = total_event_pT_hch <= 50
low_pT_events_hch = randomDataSample(sig_data['sig_hch'][mask],nEvents, random_state)

In [12]:
total_event_pT_LQ = np.sum(sig_data['sig_LQ'][:,:,0],axis = 1)
mask = total_event_pT_LQ <= 50
low_pT_events_LQ = randomDataSample(sig_data['sig_LQ'][mask],nEvents, random_state)

In [13]:
OTSCHEME = {}
OTSCHEME['normPT'] = True
OTSCHEME['balanced'] = True
OTSCHEME['noZeroPad'] = False
OTSCHEME['individualOT'] = False

In [14]:
bkg_A_event_list = np.concatenate((low_pT_events_bkg,low_pT_events_A))
bkg_A_labels = np.array([0] * nEvents + [1] * nEvents)
np.random.seed(1)
permutation = np.random.permutation(2*nEvents)
bkg_A_event_list = bkg_A_event_list[permutation]
bkg_A_labels = bkg_A_labels[permutation]
bkg_A_dm = calcOTDistance(bkg_A_event_list, bkg_A_event_list, OTSCHEME, '2D', Matrix=True)

In [15]:
neighbor_list = [5, 10, 12, 14, 16, 18, 20, 25, 30, 40, 50, 60, 70, 80, 90]
best_auc, best_k_neighbor, best_model, auc_list = kNN_with_distance_matrix(bkg_A_dm, bkg_A_labels, nEvents//2, nEvents//4, nEvents//4,neighbor_list, AUC_list=True)
print()
print(best_auc, best_k_neighbor)
print(auc_list)
del bkg_A_dm

Fitting Models: 100%|██████████| 15/15 [00:00<00:00, 311.11it/s]


0.5685897435897436 5
[0.6069594247560348, 0.5748587570621468, 0.5323574730354391, 0.5147663071391885, 0.49370826913199795, 0.4803543913713405, 0.4587827426810478, 0.4287365177195686, 0.43695428864920394, 0.45993836671802774, 0.44851052901900357, 0.4703389830508474, 0.4844632768361581, 0.4684129429892142, 0.4404211607601438]


In [16]:
bkg_h0_event_list = np.concatenate((low_pT_events_bkg,low_pT_events_h0))
bkg_h0_labels = np.array([0] * nEvents + [1] * nEvents)
np.random.seed(1)
permutation = np.random.permutation(2*nEvents)
bkg_h0_event_list = bkg_h0_event_list[permutation]
bkg_h0_labels = bkg_h0_labels[permutation]
bkg_h0_dm = calcOTDistance(bkg_h0_event_list, bkg_h0_event_list, OTSCHEME, '2D', Matrix=True)

In [17]:
neighbor_list = [5, 10, 12, 14, 16, 18, 20, 25, 30, 40, 50, 60, 70, 80, 90]
best_auc, best_k_neighbor, best_model, auc_list = kNN_with_distance_matrix(bkg_h0_dm, bkg_h0_labels, nEvents//2, nEvents//4, nEvents//4,neighbor_list, AUC_list=True)
print()
print(best_auc, best_k_neighbor)
print(auc_list)
del bkg_h0_dm

Fitting Models: 100%|██████████| 15/15 [00:00<00:00, 380.48it/s]


0.6067948717948718 10
[0.4789419619928095, 0.49897277863379563, 0.470467385721623, 0.42873651771956856, 0.44260400616332823, 0.4331022085259374, 0.42205957883923984, 0.46623009758602973, 0.42424242424242425, 0.40896250642013354, 0.42308680020544426, 0.4354134565998973, 0.45120698510529017, 0.4417051874678994, 0.4788135593220339]


In [18]:
bkg_hch_event_list = np.concatenate((low_pT_events_bkg,low_pT_events_hch))
bkg_hch_labels = np.array([0] * nEvents + [1] * nEvents)
np.random.seed(1)
permutation = np.random.permutation(2*nEvents)
bkg_hch_event_list = bkg_hch_event_list[permutation]
bkg_hch_labels = bkg_hch_labels[permutation]
bkg_hch_dm = calcOTDistance(bkg_hch_event_list, bkg_hch_event_list, OTSCHEME, '2D', Matrix=True)

In [19]:
neighbor_list = [5, 10, 12, 14, 16, 18, 20, 25, 30, 40, 50, 60, 70, 80, 90]
best_auc, best_k_neighbor, best_model, auc_list = kNN_with_distance_matrix(bkg_hch_dm, bkg_hch_labels, nEvents//2, nEvents//4, nEvents//4,neighbor_list, AUC_list=True)
print()
print(best_auc, best_k_neighbor)
print(auc_list)
del bkg_hch_dm

Fitting Models: 100%|██████████| 15/15 [00:00<00:00, 370.12it/s]


0.5234615384615384 14
[0.6114535182331793, 0.6212121212121212, 0.6302003081664098, 0.6377760657421674, 0.6201848998459167, 0.5980996404725218, 0.5654853620955316, 0.5676682074987159, 0.5621468926553672, 0.5259373394966614, 0.5147663071391885, 0.5295326142783771, 0.5132254750898818, 0.50115562403698, 0.5523882896764253]


In [20]:
bkg_LQ_event_list = np.concatenate((low_pT_events_bkg,low_pT_events_LQ))
bkg_LQ_labels = np.array([0] * nEvents + [1] * nEvents)
np.random.seed(1)
permutation = np.random.permutation(2*nEvents)
bkg_LQ_event_list = bkg_LQ_event_list[permutation]
bkg_LQ_labels = bkg_LQ_labels[permutation]
bkg_LQ_dm = calcOTDistance(bkg_LQ_event_list, bkg_LQ_event_list, OTSCHEME, '2D', Matrix=True)


In [21]:
neighbor_list = [5, 10, 12, 14, 16, 18, 20, 25, 30, 40, 50, 60, 70, 80, 90]
best_auc, best_k_neighbor, best_model, auc_list = kNN_with_distance_matrix(bkg_LQ_dm, bkg_LQ_labels, nEvents//2, nEvents//4, nEvents//4,neighbor_list, AUC_list=True)
print()
print(best_auc, best_k_neighbor)
print(auc_list)
del bkg_LQ_dm

Fitting Models: 100%|██████████| 15/15 [00:00<00:00, 540.87it/s]


0.5761538461538461 5
[0.6856702619414483, 0.6258346173600411, 0.5991268618387262, 0.579738058551618, 0.533256291730868, 0.5047508988186955, 0.5038520801232665, 0.513995891114535, 0.5202876219825372, 0.5240112994350283, 0.5254237288135593, 0.5582948125321006, 0.5394196199280945, 0.515793528505393, 0.4717514124293785]


# Mid $p_T$ range (50 GeV - 100 GeV)

In [22]:
mask = (total_event_pT_bkg >= 50) & (total_event_pT_bkg <= 100)
mid_pT_events_bkg = randomDataSample(bkg_data[mask],nEvents, random_state)

mask = (total_event_pT_A >= 50) & (total_event_pT_A <= 100)
mid_pT_events_A = randomDataSample(sig_data['sig_A'][mask],nEvents, random_state)

mask = (total_event_pT_h0 >= 50) & (total_event_pT_h0 <= 100)
mid_pT_events_h0 = randomDataSample(sig_data['sig_h0'][mask],nEvents, random_state)

mask = (total_event_pT_hch >= 50) & (total_event_pT_hch <= 100)
mid_pT_events_hch = randomDataSample(sig_data['sig_hch'][mask],nEvents, random_state)

mask = (total_event_pT_LQ >= 50) & (total_event_pT_LQ <= 100)
mid_pT_events_LQ = randomDataSample(sig_data['sig_LQ'][mask],nEvents, random_state)

In [23]:
bkg_A_event_list = np.concatenate((mid_pT_events_bkg, mid_pT_events_A))
bkg_A_labels = np.array([0] * nEvents + [1] * nEvents)
np.random.seed(1)
permutation = np.random.permutation(2*nEvents)
bkg_A_event_list = bkg_A_event_list[permutation]
bkg_A_labels = bkg_A_labels[permutation]
bkg_A_dm = calcOTDistance(bkg_A_event_list, bkg_A_event_list, OTSCHEME, '2D', Matrix=True)

neighbor_list = [3,4,5, 10, 12, 14, 16, 18, 20, 25, 30, 40, 50, 60, 70, 80, 90]
best_auc, best_k_neighbor, best_model, auc_list = kNN_with_distance_matrix(bkg_A_dm, bkg_A_labels, nEvents//2, nEvents//4, nEvents//4,neighbor_list, AUC_list=True)
print()
print(best_auc, best_k_neighbor)
print(auc_list)

del bkg_A_dm

Fitting Models: 100%|██████████| 17/17 [00:00<00:00, 746.73it/s]


0.6967948717948719 18
[0.7227786337955828, 0.743708269131998, 0.7177709296353364, 0.74653312788906, 0.7274011299435028, 0.7378017462763227, 0.7519260400616332, 0.7521828454031844, 0.7453775038520801, 0.7213662044170519, 0.7226502311248074, 0.7000513610683102, 0.6837442218798151, 0.6503595274781715, 0.6323831535695943, 0.6145351823317925, 0.6344375963020031]


In [24]:
bkg_h0_event_list = np.concatenate((mid_pT_events_bkg, mid_pT_events_h0))
bkg_h0_labels = np.array([0] * nEvents + [1] * nEvents)
np.random.seed(1)
permutation = np.random.permutation(2*nEvents)
bkg_h0_event_list = bkg_h0_event_list[permutation]
bkg_h0_labels = bkg_h0_labels[permutation]
bkg_h0_dm = calcOTDistance(bkg_h0_event_list, bkg_h0_event_list, OTSCHEME, '2D', Matrix=True)

neighbor_list = [3,4,5, 10, 12, 14, 16, 18, 20, 25, 30, 40, 50, 60, 70, 80, 90]
best_auc, best_k_neighbor, best_model, auc_list = kNN_with_distance_matrix(bkg_h0_dm, bkg_h0_labels, nEvents//2, nEvents//4, nEvents//4,neighbor_list, AUC_list=True)
print()
print(best_auc, best_k_neighbor)
print(auc_list)
del bkg_h0_dm

Fitting Models: 100%|██████████| 17/17 [00:00<00:00, 765.32it/s]


0.6241025641025642 3
[0.6905495634309193, 0.6590909090909091, 0.6706471494607088, 0.6786081150487931, 0.653954802259887, 0.6501027221366205, 0.6406009244992296, 0.6245505906522855, 0.6056753980482794, 0.6086286594761171, 0.6200564971751413, 0.6413713405238829, 0.6253210066769389, 0.5951463790446841, 0.5996404725218285, 0.6113251155624037, 0.5829481253210067]


In [25]:
bkg_hch_event_list = np.concatenate((mid_pT_events_bkg, mid_pT_events_hch))
bkg_hch_labels = np.array([0] * nEvents + [1] * nEvents)
np.random.seed(1)
permutation = np.random.permutation(2*nEvents)
bkg_hch_event_list = bkg_hch_event_list[permutation]
bkg_hch_labels = bkg_hch_labels[permutation]
bkg_hch_dm = calcOTDistance(bkg_hch_event_list, bkg_hch_event_list, OTSCHEME, '2D', Matrix=True)

neighbor_list = [3,4,5, 10, 12, 14, 16, 18, 20, 25, 30, 40, 50, 60, 70, 80, 90]
best_auc, best_k_neighbor, best_model, auc_list = kNN_with_distance_matrix(bkg_hch_dm, bkg_hch_labels, nEvents//2, nEvents//4, nEvents//4,neighbor_list, AUC_list=True)
print()
print(best_auc, best_k_neighbor)
print(auc_list)
del bkg_hch_dm

Fitting Models: 100%|██████████| 17/17 [00:00<00:00, 636.30it/s]


0.637051282051282 30
[0.5611196712891628, 0.5693374422187981, 0.561633281972265, 0.5638161273754494, 0.5557267591165896, 0.5480225988700564, 0.5364663585002567, 0.5200308166409862, 0.5318438623523368, 0.5952747817154597, 0.6142783769902413, 0.5630457113507962, 0.5670261941448382, 0.5701078582434516, 0.554185927067283, 0.5123266563944531, 0.4938366718027735]


In [ ]:
bkg_LQ_event_list = np.concatenate((mid_pT_events_bkg, mid_pT_events_LQ))
bkg_LQ_labels = np.array([0] * nEvents + [1] * nEvents)
np.random.seed(1)
permutation = np.random.permutation(2*nEvents)
bkg_LQ_event_list = bkg_LQ_event_list[permutation]
bkg_LQ_labels = bkg_LQ_labels[permutation]
bkg_LQ_dm = calcOTDistance(bkg_LQ_event_list, bkg_LQ_event_list, OTSCHEME, '2D', Matrix=True)

neighbor_list = [3,4,5, 10, 12, 14, 16, 18, 20, 25, 30, 40, 50, 60, 70, 80, 90]
best_auc, best_k_neighbor, best_model, auc_list = kNN_with_distance_matrix(bkg_LQ_dm, bkg_LQ_labels, nEvents//2, nEvents//4, nEvents//4,neighbor_list, AUC_list=True)
print()
print(best_auc, best_k_neighbor)
print(auc_list)
del bkg_LQ_dm

# Mid range 2 $p_T$(100 GeV - 150 GeV)

In [ ]:
mask = (total_event_pT_bkg >= 100) & (total_event_pT_bkg <= 150)
mid_pT_events_bkg_2 = randomDataSample(bkg_data[mask],nEvents, random_state)

mask = (total_event_pT_A >= 100) & (total_event_pT_A <= 150)
mid_pT_events_A_2 = randomDataSample(sig_data['sig_A'][mask],nEvents, random_state)

mask = (total_event_pT_h0 >= 100) & (total_event_pT_h0 <= 150)
mid_pT_events_h0_2 = randomDataSample(sig_data['sig_h0'][mask],nEvents, random_state)

mask = (total_event_pT_hch >= 100) & (total_event_pT_hch <= 150)
mid_pT_events_hch_2 = randomDataSample(sig_data['sig_hch'][mask],nEvents, random_state)

mask = (total_event_pT_LQ >= 100) & (total_event_pT_LQ <= 150)
mid_pT_events_LQ_2 = randomDataSample(sig_data['sig_LQ'][mask],nEvents, random_state)

In [ ]:
bkg_A_event_list = np.concatenate((mid_pT_events_bkg_2, mid_pT_events_A_2))
bkg_A_labels = np.array([0] * nEvents + [1] * nEvents)
np.random.seed(4)
permutation = np.random.permutation(2*nEvents)
bkg_A_event_list = bkg_A_event_list[permutation]
bkg_A_labels = bkg_A_labels[permutation]
bkg_A_dm = calcOTDistance(bkg_A_event_list, bkg_A_event_list, OTSCHEME, '2D', Matrix=True)

neighbor_list = [3,4,5, 10, 12, 14, 16, 18, 20, 25, 30, 40, 50, 60, 70, 80, 90]
best_auc, best_k_neighbor, best_model, auc_list = kNN_with_distance_matrix(bkg_A_dm, bkg_A_labels, nEvents//2, nEvents//4, nEvents//4,neighbor_list, AUC_list=True)
print()
print(best_auc, best_k_neighbor)
print(auc_list)

In [ ]:
bkg_h0_event_list = np.concatenate((mid_pT_events_bkg_2, mid_pT_events_h0_2))
bkg_h0_labels = np.array([0] * nEvents + [1] * nEvents)
np.random.seed(4)
permutation = np.random.permutation(2*nEvents)
bkg_h0_event_list = bkg_h0_event_list[permutation]
bkg_h0_labels = bkg_h0_labels[permutation]
bkg_h0_dm = calcOTDistance(bkg_h0_event_list, bkg_h0_event_list, OTSCHEME, '2D', Matrix=True)

neighbor_list = [3,4,5, 10, 12, 14, 16, 18, 20, 25, 30, 40, 50, 60, 70, 80, 90]
best_auc, best_k_neighbor, best_model, auc_list = kNN_with_distance_matrix(bkg_h0_dm, bkg_h0_labels, nEvents//2, nEvents//4, nEvents//4,neighbor_list, AUC_list=True)
print()
print(best_auc, best_k_neighbor)
print(auc_list)

In [ ]:
bkg_hch_event_list = np.concatenate((mid_pT_events_bkg_2, mid_pT_events_hch_2))
bkg_hch_labels = np.array([0] * nEvents + [1] * nEvents)
np.random.seed(4)
permutation = np.random.permutation(2*nEvents)
bkg_hch_event_list = bkg_hch_event_list[permutation]
bkg_hch_labels = bkg_hch_labels[permutation]
bkg_hch_dm = calcOTDistance(bkg_hch_event_list, bkg_hch_event_list, OTSCHEME, '2D', Matrix=True)

neighbor_list = [3,4,5, 10, 12, 14, 16, 18, 20, 25, 30, 40, 50, 60, 70, 80, 90]
best_auc, best_k_neighbor, best_model, auc_list = kNN_with_distance_matrix(bkg_hch_dm, bkg_hch_labels, nEvents//2, nEvents//4, nEvents//4,neighbor_list, AUC_list=True)
print()
print(best_auc, best_k_neighbor)
print(auc_list)

In [ ]:
bkg_LQ_event_list = np.concatenate((mid_pT_events_bkg_2, mid_pT_events_LQ_2))
bkg_LQ_labels = np.array([0] * nEvents + [1] * nEvents)
np.random.seed(4)
permutation = np.random.permutation(2*nEvents)
bkg_LQ_event_list = bkg_LQ_event_list[permutation]
bkg_LQ_labels = bkg_LQ_labels[permutation]
bkg_LQ_dm = calcOTDistance(bkg_LQ_event_list, bkg_LQ_event_list, OTSCHEME, '2D', Matrix=True)

neighbor_list = [3,4,5, 10, 12, 14, 16, 18, 20, 25, 30, 40, 50, 60, 70, 80, 90]
best_auc, best_k_neighbor, best_model, auc_list = kNN_with_distance_matrix(bkg_LQ_dm, bkg_LQ_labels, nEvents//2, nEvents//4, nEvents//4,neighbor_list, AUC_list=True)
print()
print(best_auc, best_k_neighbor)
print(auc_list)

# Mid range 3 $p_T$(150 GeV - 200 GeV)

In [ ]:
total_event_pT_bkg = np.sum(bkg_data[:, :, 0], axis=1)
mask = (total_event_pT_bkg >= 150) & (total_event_pT_bkg <= 200)
mid_pT_events_bkg_3 = randomDataSample(bkg_data[mask],nEvents, random_state)

del total_event_pT_bkg

total_event_pT_A = np.sum(sig_data['sig_A'][:,:,0],axis = 1)
mask = (total_event_pT_A >= 150) & (total_event_pT_A <= 200)
mid_pT_events_A_3 = randomDataSample(sig_data['sig_A'][mask],nEvents, random_state)

del total_event_pT_A

total_event_pT_h0 = np.sum(sig_data['sig_h0'][:,:,0],axis = 1)
mask = (total_event_pT_h0 >= 150) & (total_event_pT_h0 <= 200)
mid_pT_events_h0_3 = randomDataSample(sig_data['sig_h0'][mask],nEvents, random_state)

del total_event_pT_h0

total_event_pT_hch = np.sum(sig_data['sig_hch'][:,:,0],axis = 1)
mask = (total_event_pT_hch >= 150) & (total_event_pT_hch <= 200)
mid_pT_events_hch_3 = randomDataSample(sig_data['sig_hch'][mask],nEvents, random_state)

del total_event_pT_hch

total_event_pT_LQ = np.sum(sig_data['sig_LQ'][:,:,0],axis = 1)
mask = (total_event_pT_LQ >= 150) & (total_event_pT_LQ <= 200)
mid_pT_events_LQ_3 = randomDataSample(sig_data['sig_LQ'][mask],nEvents, random_state)

del total_event_pT_LQ

In [ ]:
bkg_A_event_list = np.concatenate((mid_pT_events_bkg_3, mid_pT_events_A_3))
bkg_A_labels = np.array([0] * nEvents + [1] * nEvents)
np.random.seed(4)
permutation = np.random.permutation(2*nEvents)
bkg_A_event_list = bkg_A_event_list[permutation]
bkg_A_labels = bkg_A_labels[permutation]
bkg_A_dm = calcOTDistance(bkg_A_event_list, bkg_A_event_list, OTSCHEME, '2D', Matrix=True)

neighbor_list = [3,4,5, 10, 12, 14, 16, 18, 20, 25, 30, 40, 50, 60, 70, 80, 90]
best_auc, best_k_neighbor, best_model, auc_list = kNN_with_distance_matrix(bkg_A_dm, bkg_A_labels, nEvents//2, nEvents//4, nEvents//4,neighbor_list, AUC_list=True)
print()
print(best_auc, best_k_neighbor)
print(auc_list)

bkg_h0_event_list = np.concatenate((mid_pT_events_bkg_3, mid_pT_events_h0_3))
bkg_h0_labels = np.array([0] * nEvents + [1] * nEvents)
np.random.seed(4)
permutation = np.random.permutation(2*nEvents)
bkg_h0_event_list = bkg_h0_event_list[permutation]
bkg_h0_labels = bkg_h0_labels[permutation]
bkg_h0_dm = calcOTDistance(bkg_h0_event_list, bkg_h0_event_list, OTSCHEME, '2D', Matrix=True)

neighbor_list = [3,4,5, 10, 12, 14, 16, 18, 20, 25, 30, 40, 50, 60, 70, 80, 90]
best_auc, best_k_neighbor, best_model, auc_list = kNN_with_distance_matrix(bkg_h0_dm, bkg_h0_labels, nEvents//2, nEvents//4, nEvents//4,neighbor_list, AUC_list=True)
print()
print(best_auc, best_k_neighbor)
print(auc_list)

bkg_hch_event_list = np.concatenate((mid_pT_events_bkg_3, mid_pT_events_hch_3))
bkg_hch_labels = np.array([0] * nEvents + [1] * nEvents)
np.random.seed(4)
permutation = np.random.permutation(2*nEvents)
bkg_hch_event_list = bkg_hch_event_list[permutation]
bkg_hch_labels = bkg_hch_labels[permutation]
bkg_hch_dm = calcOTDistance(bkg_hch_event_list, bkg_hch_event_list, OTSCHEME, '2D', Matrix=True)

neighbor_list = [3,4,5, 10, 12, 14, 16, 18, 20, 25, 30, 40, 50, 60, 70, 80, 90]
best_auc, best_k_neighbor, best_model, auc_list = kNN_with_distance_matrix(bkg_hch_dm, bkg_hch_labels, nEvents//2, nEvents//4, nEvents//4,neighbor_list, AUC_list=True)
print()
print(best_auc, best_k_neighbor)
print(auc_list)

bkg_LQ_event_list = np.concatenate((mid_pT_events_bkg_3, mid_pT_events_LQ_3))
bkg_LQ_labels = np.array([0] * nEvents + [1] * nEvents)
np.random.seed(4)
permutation = np.random.permutation(2*nEvents)
bkg_LQ_event_list = bkg_LQ_event_list[permutation]
bkg_LQ_labels = bkg_LQ_labels[permutation]
bkg_LQ_dm = calcOTDistance(bkg_LQ_event_list, bkg_LQ_event_list, OTSCHEME, '2D', Matrix=True)

neighbor_list = [3,4,5, 10, 12, 14, 16, 18, 20, 25, 30, 40, 50, 60, 70, 80, 90]
best_auc, best_k_neighbor, best_model, auc_list = kNN_with_distance_matrix(bkg_LQ_dm, bkg_LQ_labels, nEvents//2, nEvents//4, nEvents//4,neighbor_list, AUC_list=True)
print()
print(best_auc, best_k_neighbor)
print(auc_list)

In [ ]:
total_event_pT_bkg = np.sum(bkg_data[:, :, 0], axis=1)
# Plotting the histogram
plt.hist(total_event_pT_bkg, bins=1000, edgecolor='black')  # 'bins' determines the number of bins (intervals) in the histogram

# Adding title and labels
plt.title('Histogram of Data')
plt.xlabel('Value')
plt.ylabel('Frequency')
# plt.yscale('log')
plt.xlim(0,1000)

# Display the plot
plt.show()

# Mid range 4 $p_T$(200 GeV - 500 GeV)

In [ ]:
total_event_pT_bkg = np.sum(bkg_data[:, :, 0], axis=1)
mask = (total_event_pT_bkg >= 200) & (total_event_pT_bkg <= 500)
mid_pT_events_bkg_3 = randomDataSample(bkg_data[mask],nEvents, random_state)


total_event_pT_A = np.sum(sig_data['sig_A'][:,:,0],axis = 1)
mask = (total_event_pT_A >= 200) & (total_event_pT_A <= 500)
mid_pT_events_A_3 = randomDataSample(sig_data['sig_A'][mask],nEvents, random_state)


total_event_pT_h0 = np.sum(sig_data['sig_h0'][:,:,0],axis = 1)
mask = (total_event_pT_h0 >= 200) & (total_event_pT_h0 <= 500)
mid_pT_events_h0_3 = randomDataSample(sig_data['sig_h0'][mask],nEvents, random_state)


total_event_pT_hch = np.sum(sig_data['sig_hch'][:,:,0],axis = 1)
mask = (total_event_pT_hch >= 200) & (total_event_pT_hch <= 500)
mid_pT_events_hch_3 = randomDataSample(sig_data['sig_hch'][mask],nEvents, random_state)


total_event_pT_LQ = np.sum(sig_data['sig_LQ'][:,:,0],axis = 1)
mask = (total_event_pT_LQ >= 200) & (total_event_pT_LQ <= 500)
mid_pT_events_LQ_3 = randomDataSample(sig_data['sig_LQ'][mask],nEvents, random_state)

In [ ]:
bkg_A_event_list = np.concatenate((mid_pT_events_bkg_3, mid_pT_events_A_3))
bkg_A_labels = np.array([0] * nEvents + [1] * nEvents)
np.random.seed(4)
permutation = np.random.permutation(2*nEvents)
bkg_A_event_list = bkg_A_event_list[permutation]
bkg_A_labels = bkg_A_labels[permutation]
bkg_A_dm = calcOTDistance(bkg_A_event_list, bkg_A_event_list, OTSCHEME, '2D', Matrix=True)

neighbor_list = [3,4,5, 10, 12, 14, 16, 18, 20, 25, 30, 40, 50, 60, 70, 80, 90]
best_auc, best_k_neighbor, best_model, auc_list = kNN_with_distance_matrix(bkg_A_dm, bkg_A_labels, nEvents//2, nEvents//4, nEvents//4,neighbor_list, AUC_list=True)
print()
print(best_auc, best_k_neighbor)
print(auc_list)

bkg_h0_event_list = np.concatenate((mid_pT_events_bkg_3, mid_pT_events_h0_3))
bkg_h0_labels = np.array([0] * nEvents + [1] * nEvents)
np.random.seed(4)
permutation = np.random.permutation(2*nEvents)
bkg_h0_event_list = bkg_h0_event_list[permutation]
bkg_h0_labels = bkg_h0_labels[permutation]
bkg_h0_dm = calcOTDistance(bkg_h0_event_list, bkg_h0_event_list, OTSCHEME, '2D', Matrix=True)

neighbor_list = [3,4,5, 10, 12, 14, 16, 18, 20, 25, 30, 40, 50, 60, 70, 80, 90]
best_auc, best_k_neighbor, best_model, auc_list = kNN_with_distance_matrix(bkg_h0_dm, bkg_h0_labels, nEvents//2, nEvents//4, nEvents//4,neighbor_list, AUC_list=True)
print()
print(best_auc, best_k_neighbor)
print(auc_list)

bkg_hch_event_list = np.concatenate((mid_pT_events_bkg_3, mid_pT_events_hch_3))
bkg_hch_labels = np.array([0] * nEvents + [1] * nEvents)
np.random.seed(4)
permutation = np.random.permutation(2*nEvents)
bkg_hch_event_list = bkg_hch_event_list[permutation]
bkg_hch_labels = bkg_hch_labels[permutation]
bkg_hch_dm = calcOTDistance(bkg_hch_event_list, bkg_hch_event_list, OTSCHEME, '2D', Matrix=True)

neighbor_list = [3,4,5, 10, 12, 14, 16, 18, 20, 25, 30, 40, 50, 60, 70, 80, 90]
best_auc, best_k_neighbor, best_model, auc_list = kNN_with_distance_matrix(bkg_hch_dm, bkg_hch_labels, nEvents//2, nEvents//4, nEvents//4,neighbor_list, AUC_list=True)
print()
print(best_auc, best_k_neighbor)
print(auc_list)

bkg_LQ_event_list = np.concatenate((mid_pT_events_bkg_3, mid_pT_events_LQ_3))
bkg_LQ_labels = np.array([0] * nEvents + [1] * nEvents)
np.random.seed(4)
permutation = np.random.permutation(2*nEvents)
bkg_LQ_event_list = bkg_LQ_event_list[permutation]
bkg_LQ_labels = bkg_LQ_labels[permutation]
bkg_LQ_dm = calcOTDistance(bkg_LQ_event_list, bkg_LQ_event_list, OTSCHEME, '2D', Matrix=True)

neighbor_list = [3,4,5, 10, 12, 14, 16, 18, 20, 25, 30, 40, 50, 60, 70, 80, 90]
best_auc, best_k_neighbor, best_model, auc_list = kNN_with_distance_matrix(bkg_LQ_dm, bkg_LQ_labels, nEvents//2, nEvents//4, nEvents//4,neighbor_list, AUC_list=True)
print()
print(best_auc, best_k_neighbor)
print(auc_list)

# High range (500 - 1000)

In [ ]:
mask = (total_event_pT_bkg >= 500) & (total_event_pT_bkg <= 1000)
mid_pT_events_bkg_3 = randomDataSample(bkg_data[mask],nEvents, random_state)

mask = (total_event_pT_A >= 500) & (total_event_pT_A <= 1000)
mid_pT_events_A_3 = randomDataSample(sig_data['sig_A'][mask],nEvents, random_state)

mask = (total_event_pT_h0 >= 500) & (total_event_pT_h0 <= 1000)
mid_pT_events_h0_3 = randomDataSample(sig_data['sig_h0'][mask],nEvents, random_state)

mask = (total_event_pT_hch >= 500) & (total_event_pT_hch <= 1000)
mid_pT_events_hch_3 = randomDataSample(sig_data['sig_hch'][mask],nEvents, random_state)

mask = (total_event_pT_LQ >= 500) & (total_event_pT_LQ <= 1000)
mid_pT_events_LQ_3 = randomDataSample(sig_data['sig_LQ'][mask],nEvents, random_state)

In [ ]:
bkg_A_event_list = np.concatenate((mid_pT_events_bkg_3, mid_pT_events_A_3))
bkg_A_labels = np.array([0] * nEvents + [1] * nEvents)
np.random.seed(4)
permutation = np.random.permutation(2*nEvents)
bkg_A_event_list = bkg_A_event_list[permutation]
bkg_A_labels = bkg_A_labels[permutation]
bkg_A_dm = calcOTDistance(bkg_A_event_list, bkg_A_event_list, OTSCHEME, '2D', Matrix=True)

neighbor_list = [3,4,5, 10, 12, 14, 16, 18, 20, 25, 30, 40, 50, 60, 70, 80, 90]
best_auc, best_k_neighbor, best_model, auc_list = kNN_with_distance_matrix(bkg_A_dm, bkg_A_labels, nEvents//2, nEvents//4, nEvents//4,neighbor_list, AUC_list=True)
print()
print(best_auc, best_k_neighbor)
print(auc_list)

bkg_h0_event_list = np.concatenate((mid_pT_events_bkg_3, mid_pT_events_h0_3))
bkg_h0_labels = np.array([0] * nEvents + [1] * nEvents)
np.random.seed(4)
permutation = np.random.permutation(2*nEvents)
bkg_h0_event_list = bkg_h0_event_list[permutation]
bkg_h0_labels = bkg_h0_labels[permutation]
bkg_h0_dm = calcOTDistance(bkg_h0_event_list, bkg_h0_event_list, OTSCHEME, '2D', Matrix=True)

neighbor_list = [3,4,5, 10, 12, 14, 16, 18, 20, 25, 30, 40, 50, 60, 70, 80, 90]
best_auc, best_k_neighbor, best_model, auc_list = kNN_with_distance_matrix(bkg_h0_dm, bkg_h0_labels, nEvents//2, nEvents//4, nEvents//4,neighbor_list, AUC_list=True)
print()
print(best_auc, best_k_neighbor)
print(auc_list)

bkg_hch_event_list = np.concatenate((mid_pT_events_bkg_3, mid_pT_events_hch_3))
bkg_hch_labels = np.array([0] * nEvents + [1] * nEvents)
np.random.seed(4)
permutation = np.random.permutation(2*nEvents)
bkg_hch_event_list = bkg_hch_event_list[permutation]
bkg_hch_labels = bkg_hch_labels[permutation]
bkg_hch_dm = calcOTDistance(bkg_hch_event_list, bkg_hch_event_list, OTSCHEME, '2D', Matrix=True)

neighbor_list = [3,4,5, 10, 12, 14, 16, 18, 20, 25, 30, 40, 50, 60, 70, 80, 90]
best_auc, best_k_neighbor, best_model, auc_list = kNN_with_distance_matrix(bkg_hch_dm, bkg_hch_labels, nEvents//2, nEvents//4, nEvents//4,neighbor_list, AUC_list=True)
print()
print(best_auc, best_k_neighbor)
print(auc_list)

bkg_LQ_event_list = np.concatenate((mid_pT_events_bkg_3, mid_pT_events_LQ_3))
bkg_LQ_labels = np.array([0] * nEvents + [1] * nEvents)
np.random.seed(4)
permutation = np.random.permutation(2*nEvents)
bkg_LQ_event_list = bkg_LQ_event_list[permutation]
bkg_LQ_labels = bkg_LQ_labels[permutation]
bkg_LQ_dm = calcOTDistance(bkg_LQ_event_list, bkg_LQ_event_list, OTSCHEME, '2D', Matrix=True)

neighbor_list = [3,4,5, 10, 12, 14, 16, 18, 20, 25, 30, 40, 50, 60, 70, 80, 90]
best_auc, best_k_neighbor, best_model, auc_list = kNN_with_distance_matrix(bkg_LQ_dm, bkg_LQ_labels, nEvents//2, nEvents//4, nEvents//4,neighbor_list, AUC_list=True)
print()
print(best_auc, best_k_neighbor)
print(auc_list)